# IAM Boto Code Along

### Introduction

In this lesson, we'll practice working with AWS permissions.  Let's get started.

### Getting started

Let's get set up by creating a new user.  You can do so by logging into the aws console, and then typing IAM.  From there, click on `Add users`, and from there type on `codealong-user`.

<img src="./codealong-user.png" width="100%">

From here you can start granting access to the user.  For example, let's say we assign read only access.  

Search for read only access, and then you can take a look at this by clicking on the plus sign next to Amazons3ReadOnly access.

> And check the box to the left.  

<img src="./permissions.png" width="100%">

Let's also place a check mark next to IamReadOnlyAccess.

After finishing, we should see the following:

<img src="./iam-user.png" width="70%">

And then we can click on `Create User`.

After creating the user, we'll want to grant set up access to this user from the command line.  So click on the `codealong-user`, and then click on the `Security Credentials`, and then `Create Access Key` followed by CLI to specify that you want this for CLI access.

Now, from here, we can add command line access to this user with something like the following:

```bash
aws configure --profile codealong-user
```

Paste in your AWS Access Key ID, and secret access key id, as requested.  You can specify the region to be `us-east-1` or whichever region is closest to you.

Ok, now this just added a new user to your `~/.aws/credentials` file.  Type `cat ~/.aws/credentials` to view it.  You'll see that the default user is still the original user.  So change `[codealong-user]` to `[default]`, and change the original `[default]` to `[admin]` for now.

This way, when we use the boto3 library, we'll be using our codealong-user.

We can confirm this by running the `1-iam.py` file.  You should see printed out something like the following:

<img src="./iam-policies.png">

Then open up the file, and notice how we accomplished this. 

```python
import boto3
# Create an IAM client
iam_client = boto3.client('iam')

# Get the current user
response = iam_client.get_user()

# # Access the user details
username = response['User']['UserName']
print("User Name:", username)

response = iam_client.list_attached_user_policies(UserName=username)
attached_policies = response['AttachedPolicies']
print('attached_policies', attached_policies)
```

So above, we connected to the iam client, called `get_user` to get the current user, and then from there, printed out the username, which we saw was `codealong-user`.  Then we called `list_attached_user_policies` to see the policies related to that user.

> Note that if we did not grant `IAM get access` to our codealong-user above, we would not have been able to invoke the above functions regarding the iam resource.

### Seeing our First Policy

Let's finish up by looking at that first policy that we added to our user.  Remember that we added this S3 policy to the user, when we first created the user.

<img src="./permissions.png" width="100%">

Let's take a deeper look.

```python
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:Get*",
                "s3:List*",
                "s3-object-lambda:Get*",
                "s3-object-lambda:List*"
            ],
            "Resource": "*"
        }
    ]
}
```

Ok, so notice that there are only two outer keys: `Version` and `Statement`.  Version is the API version of AWS that we are using: here the 2012 API, which is most recent.  And then `Statement` takes a list of dictionaries, with each dictionary defining a statement.  

In the above, we are granting an `Allow` (in other words permission) to our IAM user, `codealong-user` to invoke the actions of listing and getting. The `*` in `s3:Get*` means that it has access to all of the related Get actions.  Then we can see the `Resource`.  This is where can indicate which resources we have this access to.  Above, we are granting the specified actions to any S3 resource.   

> You'll also see that we have the actions of `s3-object-lambda:Get*`.  This means that we can also set up s3 to invoke a lambda function when that s3 object is retrieved.  (It's not a big deal).

Finally, ote that the above is an *IAM policy*.  This means that it is a policy attached to a user or group.  As we'll see later, we can also attach policies on the resource itself, like an S3 bucket, that specifies which users are allowed to access it.

In the next lesson, we'll work with this s3 access.  But for now, that's enough.

### Summary

In this lesson, we moved through setting up permissions for a new iam user.  We reviewed setting up these permissions, and adding a separate profile to access from the command line.

We then saw how we can view the current user through boto, and the related permissions through boto.  

Finally, we moved through understanding the s3 policy attached to our user.

### Resources

[Iam and bucket policies](https://aws.amazon.com/blogs/security/iam-policies-and-bucket-policies-and-acls-oh-my-controlling-access-to-s3-resources/)

[IAM Policies](https://start.jcolemorrison.com/aws-iam-policies-in-a-nutshell/)